In [ ]:
!pip install fosforml

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [3]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

In [4]:
df = 'ATTRITION_PREDICTIVE_DATA'

In [5]:
sf_df = my_session.sql("select * from {}".format(df))

In [6]:
import pandas as pd
pandas_df = sf_df.to_pandas()

In [7]:
print(pandas_df.isnull().sum())

EMPLOYEE_ID                      0
TENURE_MONTHS                    0
BIRTH_YEAR                       0
AGE                              0
SENIORITY                        0
SCHOOL_ENDDATE                   0
JOB_STARTDATE                    0
JOB_ENDDATE                  90076
CITY                             0
DISTANCE                         0
DEGREE_CLEAN                     0
ETHNICITY                        0
MARITAL_STATUS                   0
ROLE                             0
COMPANY_NAME                     0
ORGANIZATION_TYPE                0
ORGANIZATION_OWNERSHIP           0
STATE                            0
COUNTRY                          0
GENDER                           0
WORK_LIFE_BALANCE                0
BUSINESS_TRAVEL                  0
ENVIRONMENT_SATISFACTION         0
SALARY                           0
JOB_SATISFACTION                 0
PERCENTAGE_SALARY_HIKE           0
PERFORMANCE_RATING               0
OVER_TIME                        0
RELATIONSHIP_SATISFA

In [8]:
Original_df = pandas_df.dropna()

In [9]:
print(Original_df.isnull().sum())

EMPLOYEE_ID                  0
TENURE_MONTHS                0
BIRTH_YEAR                   0
AGE                          0
SENIORITY                    0
SCHOOL_ENDDATE               0
JOB_STARTDATE                0
JOB_ENDDATE                  0
CITY                         0
DISTANCE                     0
DEGREE_CLEAN                 0
ETHNICITY                    0
MARITAL_STATUS               0
ROLE                         0
COMPANY_NAME                 0
ORGANIZATION_TYPE            0
ORGANIZATION_OWNERSHIP       0
STATE                        0
COUNTRY                      0
GENDER                       0
WORK_LIFE_BALANCE            0
BUSINESS_TRAVEL              0
ENVIRONMENT_SATISFACTION     0
SALARY                       0
JOB_SATISFACTION             0
PERCENTAGE_SALARY_HIKE       0
PERFORMANCE_RATING           0
OVER_TIME                    0
RELATIONSHIP_SATISFACTION    0
CHURN                        0
dtype: int64


In [11]:
df = Original_df[["DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE","CHURN"]]

In [12]:
df

,DISTANCE,DEGREE_CLEAN,ROLE,COMPANY_NAME,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,JOB_ENDDATE,CHURN
0,<2 miles,Bachelors Degree,Accounting Manager,Horizon Financials,Financial Services,Private,2019-08-01,1
1,<2 miles,Undergraduate Diploma,Compliance Manager,Vertex Dynamics,Aerospace,Private,2021-06-01,1
3,<2 miles,Bachelors Degree,Process Assistant,Horizon Financials,Financial Services,Private,2017-07-01,1
4,<2 miles,Bachelors Degree,Operations Manager,BrightStart Ventures,E-commerce,Private,2019-08-01,1
6,>10 miles,Other,Operations Manager,BrightStart Ventures,E-commerce,Private,2018-07-01,1
...,...,...,...,...,...,...,...,...
299995,>10 miles,Masters Degree,Corporate Counsel,BridgePoint Capital,Investment,Public,2023-05-01,1
299996,>10 miles,Masters Degree,Compliance Manager,SecureTech Solutions,Technology,Public,2023-05-01,1
299997,>10 miles,Masters Degree,Process Assistant,GreenWorld Foundation,Environmental,Non-Profit,2023-05-01,1
299998,>10 miles,Masters Degree,Data Scientist,Pinnacle Technologies,Technology,Public,2023-05-01,1


In [16]:
New_df = df.groupby("DISTANCE","DEGREE_CLEAN","ROLE","COMPANY_NAME","ORGANIZATION_TYPE","ORGANIZATION_OWNERSHIP","JOB_ENDDATE").agg('sum')

ValueError: No axis named DEGREE_CLEAN for object type DataFrame

In [13]:
df['JOB_ENDDATE'] = pd.to_datetime(df['JOB_ENDDATE'])
monthly_data = df.resample('M', on='JOB_ENDDATE').sum()

In [14]:
monthly_data

,DISTANCE,DEGREE_CLEAN,ROLE,COMPANY_NAME,ORGANIZATION_TYPE,ORGANIZATION_OWNERSHIP,CHURN
JOB_ENDDATE,,,,,,,
2010-02-28,<2 miles<2 miles<2 miles<2 miles<2 miles<2 mil...,Associates DegreeAssociates DegreeAssociates D...,UX DesignerPolicy AdvisorMachine Learning Scie...,TechNova Inc.Innovatech Ltd.AlphaRetail Ltd.Al...,TechnologyTechnologyRetailRetailEnvironmental ...,PublicPrivatePublicPublicPrivatePublicPublicNo...,158
2010-03-31,<2 miles<2 miles<2 miles<2 miles<2 miles<2 mil...,Associates DegreeAssociates DegreeAssociates D...,Policy AdvisorOperations ManagerVisual Designe...,FutureWave Corp.MetroHealth SystemsGreenWorld ...,ElectronicsHealthcareEnvironmentalE-commerceTe...,PublicNon-ProfitNon-ProfitPrivatePublicPrivate...,137
2010-04-30,<2 miles<2 miles<2 miles<2 miles<2 miles<2 mil...,Bachelors DegreeBachelors DegreeBachelors Degr...,Technical Support AssociateVisual DesignerTech...,SecureTech SolutionsBridgePoint CapitalAlphaRe...,TechnologyInvestmentRetailFinancial ServicesTe...,PublicPublicPublicCooperativePublicState-Owned...,141
2010-05-31,<2 miles<2 miles<2 miles<2 miles<2 miles<2 mil...,Masters DegreeMasters DegreeMasters DegreeMast...,Account ExecutiveProduct ManagerData Scientist...,GreenByte SolutionsHorizon FinancialsPinnacle ...,IT ServicesFinancial ServicesTechnologyEnviron...,PublicPrivatePublicPrivatePublicPublicCooperat...,261
2010-06-30,<2 miles<2 miles<2 miles<2 miles<2 miles<2 mil...,Associates DegreeAssociates DegreeAssociates D...,Data Center TechnicianProduct ManagerMachine L...,BlueSky EnergyBridgePoint CapitalRuralCooperat...,EnergyInvestmentFinancial ServicesEnergyEnviro...,PrivatePublicCooperativePrivatePrivatePublicPr...,124
...,...,...,...,...,...,...,...
2024-07-31,>10 miles>10 miles>10 miles>10 miles>10 miles>...,OtherOtherOtherOtherOtherOtherOtherOtherOtherO...,Marketing ManagerWarehouse AssociateRecruiterD...,BridgePoint CapitalTechNova Inc.Innovatech Ltd...,InvestmentTechnologyTechnologyIT ServicesAeros...,PublicPublicPrivatePublicPrivatePublicPrivateP...,2644
2024-08-31,>10 miles>10 miles>10 miles>10 miles>10 miles>...,OtherOtherOtherOtherOtherOtherOtherOtherOtherO...,Operations ManagerCustomer Service AssociateCo...,Horizon FinancialsStateGrid UtilitiesPinnacle ...,Financial ServicesUtilitiesTechnologyTransport...,PrivateState-OwnedPublicPrivateCooperativeJoin...,1988
2024-09-30,5-10 miles>10 miles<2 miles<2 miles>10 miles<2...,Associates DegreeAssociates DegreeUndergraduat...,Creative DirectorAccounting ManagerMarketing M...,BlueSky EnergyBrightStart VenturesUrbanGreen L...,EnergyE-commerceEnvironmental ServicesE-commer...,PrivatePrivatePrivatePrivatePublicNon-ProfitPu...,1269
